### Prompt Engineering

- We'll use LangChain.
- Langchain is a framework for developing applications powered by large language models.

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import os

In [7]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "API_KEY"
    print('API_KEY set properly')


In [8]:
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)

#### Zero-Shot Prompting
- Asking the model to perform a task without giving any example.
- When you want quick result and the task is simple or well-known
- Example:
    - Translate the following English sentence to French: "How are you?"


In [9]:
prompt = PromptTemplate.from_template("Translate the sentences to French:{sentence}")
chain = LLMChain(llm=llm,prompt=prompt)
print(chain.run("How are you?"))

The translation of "How are you?" in French is "Comment ça va ?" or simply "Ça va ?" for a more informal approach. If you want to be more formal, you can say "Comment allez-vous ?"


#### One-Shot Prompting
- Providing one example of the task along with the instruction.
- Useful when the model needs a little context or format guidance.
- Example
    - prompt = """Translate the following sentence to French.
        Example: "I love apples." → "J'aime les pommes."
        Now translate: "Good morning."""
        


In [17]:
template = """Parse a customer's pizza order into a valid JSON:
Example: 
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
{{
    "size" : "small",
    "type" : "normal",
    "ingredients" : [["cheese", "tomato sauce", "pepperoni"]]
}}
Now Parse this customer order: {sentence}
Respond only with the JSON object.
JSON Response:
"""
prompt = PromptTemplate(input_variables = ["sentence"],template=template)
chain = LLMChain(llm=llm,prompt=prompt)
response = chain.run("I would like a large pizza, with the first half cheese and mozzarella. And the other tomato sauce, ham and pineapple.")
print(response)

{
    "size": "large",
    "type": "normal",
    "ingredients": [["cheese", "mozzarella"], ["tomato sauce", "ham", "pineapple"]]
}


#### Few-Shot Prompting
- Providing multiple examples to show model a pattern.
- Works well when the task requires context or formatting that zero-shot fails to capture.
- examples = {
    "I am a student" : "আমি একজন ছাত্র।"
    "The sky is blue" : "আকাশ নীল"
    "They are eating.": "তারা খাচ্ছে।"
}

In [22]:
examples = [
    ("I am a student.", "আমি একজন ছাত্র।"),
    ("The sky is blue", "আকাশ নীল"),
    ("They are eating.", "তারা খাচ্ছে।")
]
example_str = "\n".join([f"{eng}->{ben}" for eng, ben in examples])
template = f"""Translate these sentences:
{example_str}
translate: {{sentence}}"""
Prompt = PromptTemplate.from_template(template)
chain = LLMChain(llm=llm,prompt=Prompt)
response = chain.run("She is happy.")
response1 = chain.run("He is a briliant student")
print(response)
print("*"*10)
print(response1)

She is happy. -> সে খুশি।
**********
He is a brilliant student. -> তিনি একজন উজ্জ্বল ছাত্র।


#### Chain-of-Thought Prompting
- Asking the model to explain its reasoning step-by-step before giving the final answer.
- Effective in reasoning-based tasks like math, logic puzzles, and complex decision making.


In [23]:
prompt = PromptTemplate.from_template(
    "Question: {question}\nThink step-by-step."
)
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run("If there are 5 pens and you take away 2, how many do you have?")
print(response)


Let's break it down step-by-step:

1. **Initial Count**: You start with 5 pens.
2. **Taking Away**: You take away 2 pens from the initial 5.
3. **Your Count**: Since you took away 2 pens, you now have those 2 pens in your possession.

So, the answer is that you have 2 pens.


#### Role-Based Prompting
- Asking the model to act in a specific role or persona
- Ideal for simulations, interviews, support bots and educational uses.

In [24]:
template = """You are a senior Data Scientist. Explain decision trees in simple terms to a beginner in 3 lines."""
prompt = PromptTemplate.from_template(template)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({}))

Decision trees are a type of model used for making decisions based on data. They work by splitting data into branches based on certain criteria, leading to a final decision or prediction at the leaves. Think of it like a flowchart that helps you choose the best option by asking a series of yes/no questions.


#### Instruction Based prompting
- Using natural language instructions to guide the model's behaviour.
- Supported heavily in models like GPT-3.5-turbo and GPT-4

In [25]:
prompt = PromptTemplate.from_template(
    "Summarize the following paragraph in 2 bullet points:\n\n{paragraph}"
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run("Artificial Intelligence is transforming the healthcare industry..."))


- Artificial Intelligence is significantly changing the healthcare industry by enhancing diagnostics, treatment plans, and patient care.
- AI technologies are improving efficiency, reducing costs, and enabling personalized medicine in healthcare settings.


##### There are different other prompting techniques.
- ReAct promting (Reasoning + Actions)
- Multimodal Prompting (Langchain with GPT-4-vision)
- Prompt chaning
- RAG (Retrieval-Augmented Generation)